(sor-method-section)=

# The Successive Over Relaxation (SOR) method

The **Successive Over Relaxation (SOR)** method improves on the convergence rate of the [Gauss-Seidel](gauss-seidel-method-section) method by applying a weighting factor to the updated estimates to adjust the extent of the change. Let $\omega$ be a **relaxation parameter** in the range $[0,2]$ then the SOR method is

```{prf:definition} The Successive Over Relaxation (SOR) method
:label: sor-method-definition

The SOR method for solving a system of linear equations of the form $A \vec{x} = \vec{b}$ is

$$ x_i^{(k+1)} =(1 - \omega) x_i^{(k)} + \frac{\omega}{a_{ii} }\left(b_i - \sum_{j=1}^{i-1} a_{ij} x_j^{(k+1)} -\sum_{j=i+1}^n a_{ij} x_j^{(k)} \right), \qquad i = 1, \ldots, n. $$(sor-method-equation)

where $\omega \in [0, 2]$.
```

## The iteration matrix for the SOR method

The iteration matrix for the SOR method is derived by writing the coefficient matrix of the linear system $A\vec{x}=\vec{b}$ using

$$ A = L+\left(1-\frac{1}{\omega }\right)D+\frac{1}{\omega }D+U. $$

Substituting into the linear system $A \vec{x} = \vec{b}$ and rearranging

$$ \begin{align*}
    \left(L+\left(1-\frac{1}{\omega }\right)D+\frac{1}{\omega }D+U\right)\vec{x}&=\vec{b}\\
    (D+\omega L)\vec{x}+((\omega -1)D+\omega U)\vec{x}&=\omega \vec{b}\\
    (D+\omega L)\vec{x}&=((1-\omega )D-\omega U)\vec{x}+\omega \vec{b}\\
    \vec{x}&=(D+\omega L)^{-1} ((1-\omega )D-\omega U)\vec{x}+(D+\omega L)^{-1} \omega \vec{b}.
\end{align*} $$

So the matrix form of the SOR method is

$$ \vec{x}^{(k+1)} =(D+\omega L)^{-1} ((1-\omega )D-\omega U)\vec{x}^{(k)} +(D+\omega L)^{-1} \omega \vec{b} $$

and the iteration matrix is

$$ T_{SOR} =(D+\omega L)^{-1} ((1-\omega )D-\omega U). $$(sor-method-iteration-matrix-equation)

## Under and over relaxation

The value of $\omega$ affects the rate of convergence of the SOR method and is determined by how the Gauss-Seidel method converges to the exact solution. Consider the diagrams in {numref}`over-relaxation-figure` and {numref}`under-relaxation-figure` below that shows the two types of convergence of the Gauss-Seidel method. When we have monotonic convergence where each successive iteration approaches the exact solution from a single side ({numref}`over-relaxation-figure`), the rate of convergence is improved by using $\omega > 1$ so that the change in subsequent approximations is increased. When we have oscillatory convergence where successive iterations overshoot the exact solution ({numref}`under-relaxation-figure`) the rate of convergence is improved by using $\omega < 1$ so that the change in the subsequent approximation is decreased.

```{figure} ../_images/sor_1.svg
:name: over-relaxation-figure
:width: 400

The estimated solution approaches the exact solution monotonically.
```

```{figure} ../_images/sor_2.svg
:name: under-relaxation-figure
:width: 400

The estimated solution oscillates about the exact solution.
```

(optimum-relaxation-parameter-section)=
## Optimum value of the relaxation parameter

The optimum value of $\omega$ will be the one that minimises the spectral radius of the iteration matrix. The iteration matrix will depend on the value of $\omega$. One way to estimate the optimum value of $\omega$ is to calculate $\rho(T_{SOR})$ for values in the range $\omega \in [0, 2]$ and choose the value of $\omega$ which minimises this value. This has been done for the system of linear equations from {prf:ref}`jacobi-method-example` and the plot is shown in {numref}`optimum-relaxation-parameter-figure`.

```{glue:figure} optimum_omega_plot
:name: optimum-relaxation-parameter-figure
:figwidth: 600

Plot of the spectral radius $\rho(T_{SOR})$ of the iteration matrix for the SOR method when used to solve the system of linear equations from {prf:ref}`jacobi-method-example`.
```

So an estimation of the optimum value of the relaxation parameters is $\omega = 1.25$. If the coefficient matrix $A$ has real eigenvalues we can use the following theorem to calculate the exact value of $\omega$. 


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def sor_iteration_matrix(A, omega):
    L = np.tril(A, -1)
    U = np.triu(A, 1)
    D = A - L - U
    return np.dot(np.linalg.inv(D + omega * L), ((1 - omega) * D - omega * U))


# Define coefficient matrix
A = np.array([[4, 3, 0], [3, 4, -1], [0, -1, 4]])

# Loop through values of omega and calculate the spectral radius of the iteration matrix
omega = np.arange(0, 2 + 1e-4, 1e-4)
rho = np.zeros(omega.shape)
for i in range(len(omega)):
    T_SOR = sor_iteration_matrix(A, omega[i])
    rho[i] = max(abs(np.linalg.eigvals(T_SOR)))
    
# Plot spectral radii
fig, ax = plt.subplots()
plt.plot(omega, rho, "b")
plt.xlabel("$\omega$", fontsize=14)
plt.ylabel(r"$\rho(T_{SOR})$")
plt.show()

from myst_nb import glue
glue("optimum_omega_plot", fig, display=False)


```{prf:theorem} Optimum relaxation parameter
:label: optimum-relaxation-parameter-theorem

If a system of linear equations of the form $A\vec{x}=\vec{b}$ has a coefficient matrix $A$ with all real eigenvalues then the optimum relaxation parameter for the SOR method can be calculated using

$$ \omega_{opt} = 1+{\left(\frac{\rho (T_J )}{1+\sqrt{1-\rho (T_J )^2 }}\right)}^2, $$(optimum-relaxation-parameter-equation)

where $\rho(T_J)$ is the spectral radius of the iteration matrix of the Jacobi method calculation using equation {eq}`jacobi-method-iteration-matrix-equation`.
```

`````{prf:example}
:label: optimum-relaxation-parameter-example

Determine the optimum relaxation parameter for the SOR method when applied to the system of linear equations from {prf:ref}`jacobi-method-example` using equation {eq}`optimum-relaxation-parameter-equation`.

````{dropdown} Solution (click to show)

We saw in {prf:ref}`convergence-example` that the spectral radius of the iteration matrix for the Jacobi method for this system is $\rho(T_J )=0.7906$.

Using equation {eq}`optimum-relaxation-parameter-equation`

$$ \begin{align*}
    \omega = 1 + \left( \frac{0.7906}{1 + \sqrt{1 - 0.7906)^2}} \right)^2 \approx 1.2404.
\end{align*} $$
````
`````

`````{prf:example}
:label: sor-method-example

Solve the system of linear equations from from {prf:ref}`jacobi-method-example` using the SOR method with $\omega = 1.24$ and an accuracy tolerance of $tol = 10^{-4}$

$$ \begin{align*}
    4x_1 +3x_2 &=-2,\\
    3x_1 +4x_2 -x_3 &=-8,\\
    -x_2 +4x_3 &=14.
\end{align*} $$

```{dropdown} Solution (click to show)

The SOR method for this system is

$$ \begin{align*}
    x_{1}^{(k+1)} &= (1 - \omega)x_{1}^{(k)} + \frac{\omega}{4} \left( -2 - 3 x_{2}^{(k)} \right), \\
    x_{2}^{(k+1)} &= (1 - \omega)x_{2}^{(k)} + \frac{\omega}{4} \left( -8 - 3 x_{1}^{(k+1)} + x_{3}^{(k)} \right), \\
    x_{3}^{(k+1)} &= (1 - \omega)x_{3}^{(k)} + \frac{\omega}{4} \left( 14 + x_{2}^{(k+1)} \right).
\end{align*} $$

Using starting values of $\vec{x} = \vec{0}$. Calculating the first iteration

$$ \begin{align*}
    x_{1}^{(1)} &= (1 - 1.24)(-0.5) \frac{1.24}{4} \left( -2 - 3 (-1.625) \right) = -0.5, \\
    x_{2}^{(1)} &= (1 - 1.24)(-1.625) \frac{1.24}{4} \left( -8 - 3 (-0.5) + -3.09375 \right) = -1.625, \\
    x_{3}^{(1)} &= (1 - 1.24)(3.09375) \frac{1.24}{4} \left( 14 + 1.625 \right) = 3.09375.
\end{align*} $$

Calculate the residual

$$ \begin{align*}
    \vec{r}^{(1)} &= \vec{b} - A \vec{x}^{(1)} = 
    \begin{pmatrix} -2 \\ -8 \\ 14 \end{pmatrix} -
    \begin{pmatrix} 4 & 3 & 0 \\ 3 & 4 & -1 \\ 0 & -1 & 4 \end{pmatrix}
    \begin{pmatrix} -0.5 \\ -1.625 \\ 3.09375 \end{pmatrix} \\
    &=
    \begin{pmatrix} 4.875 \\ 3.09375 \\ 0.0 \end{pmatrix}.
\end{align*} $$

Since $\max(\| \vec{r}^{(1)} \|) = 4.875 > 10^{-4}$ we continue iterating. Calculating the second iteration

$$ \begin{align*}
    x_{1}^{(2)} &= (1 - 1.24)(0.71875) \frac{1.24}{4} \left( -2 - 3 (-1.765625) \right) = 0.71875, \\
    x_{2}^{(2)} &= (1 - 1.24)(-1.765625) \frac{1.24}{4} \left( -8 - 3 (0.71875) + -3.05859375 \right) \\
    &= -1.765625, \\
    x_{3}^{(2)} &= (1 - 1.24)(3.05859375) \frac{1.24}{4} \left( 14 + 1.765625 \right) = 3.05859375.
\end{align*} $$

Calculate the residual

$$ \begin{align*}
    \vec{r}^{(2)} &= \vec{b} - A \vec{x}^{(1)} = 
    \begin{pmatrix} -2 \\ -8 \\ 14 \end{pmatrix} -
    \begin{pmatrix} 4 & 3 & 0 \\ 3 & 4 & -1 \\ 0 & -1 & 4 \end{pmatrix}
    \begin{pmatrix} 0.71875 \\ -1.765625 \\ 3.05859375 \end{pmatrix} \\
    &=
    \begin{pmatrix} 0.421875 \\ -0.03515625 \\ 0.0 \end{pmatrix}.
\end{align*} $$

Since $\max(| \vec{r}^{(2)} |) = 0.421875 > 10^{-4}$ we continue iterating. The SOR method was iterated until $\max(|\vec{r}|) < 10^{-4}$ and the iteration values are given in the table below. Note that the SOR method took 9 iterations to achieve convergence to $tol=10^{-4}$ whereas the {prf:ref}`Jacobi method<jacobi-method-example>` took 49 iterations and the {prf:ref}`Gauss-Seidel method<gauss-seidel-method-example>` took 19 iterations to achieve the same accuracy.

| $k$ |  $x_{1}$  |  $x_{2}$  |  $x_{3}$  | $\max(\| \vec{r} \|)$ |
|:---:|:---------:|:---------:|:---------:|:---------:|
|   0 |  0.000000 |  0.000000 |  0.000000 | 14.000000 |
|   1 | -0.620000 | -1.903400 |  3.749946 |  6.19e+00 |
|   2 |  1.298962 | -2.068735 |  2.798705 |  9.90e-01 |
|   3 |  0.992173 | -2.038626 |  3.036337 |  2.14e-01 |
|   4 |  1.037801 | -2.014620 |  2.986747 |  1.07e-01 |
|   5 |  1.004524 | -2.004807 |  3.001690 |  1.16e-02 |
|   6 |  1.003385 | -2.001470 |  2.999139 |  9.13e-03 |
|   7 |  1.000555 | -2.000430 |  3.000073 |  9.29e-04 |
|   8 |  1.000267 | -2.000122 |  2.999944 |  7.01e-04 |
|   9 |  1.000050 | -2.000034 |  3.000003 |  9.65e-05 |

```
`````

In [3]:
import numpy as np

def sor_table(A, b, omega, tol=1e-6):
    n = len(b)
    x = np.zeros(n)
    maxiter = 100
    r = b - np.dot(A, x)
    
    string = f"| $k$ | "
    for i in range(n):
        string += f"  $x_{i}$   | "
    string += r" $\max{|\vec{r}|}$ |"
    print(string)

    string = f"| {0:3d} | "
    for i in range(n):
        string += f"{x[i]:9.6f} | "
    string += f"{max(abs(r)):0.2e} |"
    print(string)

    for k in range(maxiter):
        for i in range(n):
            sum_ = 0
            for j in range(n):
                if i != j:
                    sum_ += A[i,j] * x[j]
        
            x[i] = (1 - omega) * x[i] + omega * (b[i] - sum_) / A[i,i]
            
        r = b - np.dot(A, x)

        string = f"| {k:3d} | "
        for i in range(n):
            string += f"{x[i]:9.6f} | "
        string += f"{max(abs(r)):0.2e} |"
        print(string)

        if max(abs(r)) < tol:
            break
    
    return x


def sor_latex(A, b, omega, tol=1e-6):
    n = len(b)
    x = np.zeros(n)
    maxiter = 100
    ordinal = ["first", "second", "third"]
    print("The SOR method for this system is")
    print()
    print(r"$$ \begin{align*}")
    for i in range(n):
        string = rf"    x_{{{i+1}}}^{{(k+1)}} &= (1 - \omega)x_{{{i+1}}}^{{(k)}} + \frac{{\omega}}{{{A[i,i]}}} \left( {b[i]}"
        for j in range(n):
            if i == j:
                continue
            if A[i,j] == 1:
                string += rf" - x_{{{j+1}}}"
            elif A[i,j] == -1:
                string += rf" + x_{{{j+1}}}"
            elif A[i,j] < 0:
                string += rf" + {-A[i,j]} x_{{{j+1}}}"
            elif A[i,j] > 0:
                string += rf" - {A[i,j]} x_{{{j+1}}}"
            if j < i and A[i,j] != 0:
                string += rf"^{{(k+1)}}"
            elif j > i and A[i,j] != 0:
                string += rf"^{{(k)}}"
        
        if i == n - 1:
            string += r" \right)."
        else:
            string += r" \right), \\"
            
        print(string)
    print(r"\end{align*} $$")
    print()
    print("Using starting values of $\vec{x} = \vec{0}$. ", end="")
        
    for k in range(2):
        for i in range(n):
            s = b[i]
            for j in range(n):
                if i != j:
                    s -= A[i,j] * x[j]

            x[i] = s / A[i,i]
            r = b - np.dot(A, x)
        
        print(rf"Calculating the {ordinal[k]} iteration")
        print()
        print(r"$$ \begin{align*}")
        for i in range(n):
            string = rf"    x_{{{i+1}}}^{{({k+1})}} &= (1 - {omega})({x[i]}) \frac{{{omega}}}{{{A[i,i]}}} \left( {b[i]}"
            for j in range(n):
                if i == j:
                    continue
                if A[i,j] == 1:
                    string += rf" - {x[j]}"
                elif A[i,j] == -1:
                    string += rf" + {-x[j]}"
                elif A[i,j] < 0:
                    string += rf" + {-A[i,j]} ({x[j]})"
                elif A[i,j] > 0:
                    string += rf" - {A[i,j]} ({x[j]})"

            if i == n - 1:
                string += rf" \right) = {x[i]}."
            else:
                string += rf" \right) = {x[i]}, \\"

            print(string)
        
        print(r"\end{align*} $$")
        print()
        print("Calculate the residual")
        print()
        print(r"$$ \begin{align*}")
        print(rf"    \vec{{r}}^{{({k+1})}} = \vec{{b}} - A \vec{{x}}^{{(1)}} = ")
        string = r"    \begin{pmatrix}"
        for i in range(n):
            string += rf" {b[i]}"
            if i < n - 1:
                string += r" \\"
        string += r" \end{pmatrix} -"
        print(string)

        string = r"    \begin{pmatrix}"
        for i in range(n):
            for j in range(n):
                string += rf" {A[i,j]}"
                if j < n - 1:
                    string += r" &"
            if i < n - 1:
                string += r" \\"
        string += r" \end{pmatrix}"
        print(string)

        string = r"    \begin{pmatrix}"
        for i in range(n):
            string += rf" {x[i]}"
            if i < n - 1:
                string += r" \\"
        string += r" \end{pmatrix} ="
        print(string)

        string = r"    \begin{pmatrix}"
        for i in range(n):
            string += rf" {r[i]}"
            if i < n - 1:
                string += r" \\"
        string += r" \end{pmatrix}."
        print(string)
        print(r"\end{align*} $$")
        print()
        print(rf"Since $\max(| \vec{{r}}^{{({k+1})}} |) = {max(abs(r))} > 10^{{-4}}$ we continue iterating. ", end="")
        
# Define linear system
A = np.array([[4, 3, 0], [3, 4, -1], [0, -1, 4]])
b = np.array([-2, -8, 14])

# Solve linear system
omega = 1.24
sor_table(A, b, omega)
sor_latex(A, b, omega)

| $k$ |   $x_0$   |   $x_1$   |   $x_2$   |  $\max{|\vec{r}|}$ |
|   0 |  0.000000 |  0.000000 |  0.000000 | 1.40e+01 |
|   0 | -0.620000 | -1.903400 |  3.749946 | 6.19e+00 |
|   1 |  1.298962 | -2.068735 |  2.798705 | 9.90e-01 |
|   2 |  0.992173 | -2.038626 |  3.036337 | 2.14e-01 |
|   3 |  1.037801 | -2.014620 |  2.986747 | 1.07e-01 |
|   4 |  1.004524 | -2.004807 |  3.001690 | 1.16e-02 |
|   5 |  1.003385 | -2.001470 |  2.999139 | 9.13e-03 |
|   6 |  1.000555 | -2.000430 |  3.000073 | 9.29e-04 |
|   7 |  1.000267 | -2.000122 |  2.999944 | 7.01e-04 |
|   8 |  1.000050 | -2.000034 |  3.000003 | 9.65e-05 |
|   9 |  1.000020 | -2.000009 |  2.999996 | 5.10e-05 |
|  10 |  1.000004 | -2.000003 |  3.000000 | 8.17e-06 |
|  11 |  1.000001 | -2.000001 |  3.000000 | 3.58e-06 |
|  12 |  1.000000 | -2.000000 |  3.000000 | 6.32e-07 |
The SOR method for this system is

$$ \begin{align*}
    x_{1}^{(k+1)} &= (1 - \omega)x_{1}^{(k)} + \frac{\omega}{4} \left( -2 - 3 x_{2}^{(k)} \right), \\
    x_{2}^

## Code

The code below defines a function called `sor()` which solves a linear system of equations of the for $A \vec{x} = \vec{b}$ using the SOR method with a relaxation parameter $\omega$.

`````{tab-set}
````{tab-item} Python
```python
def sor(A, b, omega, tol):
    n = len(b)
    x = np.zeros(n)
    for k in range(100):
        for i in range(n):
            sum_ = 0
            for j in range(n):
                if i != j:
                    sum_ += A[i,j] * x[j]
        
            x[i] = (1 - omega) * x[i] + omega * (b[i] - sum_) / A[i,i]
            
        r = b - np.dot(A, x)
        if max(abs(r)) < tol:
            break
    
    return x
```
````

````{tab-item} MATLAB
```matlab
function x = sor(A, b, omega, tol)

n = length(b);
x = zeros(n, 1);
for k = 1 : 100
    for i = 1 : n
        sum_ = 0;
        for j = 1 : n
            if j ~= i
                sum_ = sum_ + A(i,j) * x(j);
            end
        end
        x(i) = (1 - omega) * x(i) + omega * (b(i) - sum_) / A(i,i);
    end
    r = b - A * x;
    if max(abs(r)) < tol
        break
    end
end

end
```
`````

In [5]:
import numpy as np

def sor(A, b, omega, tol=1e-6):
    n = len(b)
    x = np.zeros(n)
    for k in range(100):
        for i in range(n):
            sum_ = 0
            for j in range(n):
                if i != j:
                    sum_ += A[i,j] * x[j]
        
            x[i] = (1 - omega) * x[i] + omega * (b[i] - sum_) / A[i,i]
            
        r = b - np.dot(A, x) 
        if max(abs(r)) < tol:
            break
    
    return x


 # Define linear system
A = np.array([[4, 3, 0], [3, 4, -1], [0, -1, 4]])
b = np.array([-2, -8, 14])

# Solve linear system
omega = 1.24
x = sor(A, b, omega, tol=1e-4)
print(f"x = {x}")

0 [-0.62     -1.9034    3.749946]
1 [ 1.298962   -2.0687354   2.79870499]
2 [ 0.99217304 -2.03862589  3.03633678]
3 [ 1.03780055 -2.01461989  2.98674701]
4 [ 1.00452437 -2.00480732  3.00169045]
5 [ 1.00338496 -2.00147021  2.99913853]
6 [ 1.00055491 -2.00043027  3.00007337]
7 [ 1.00026697 -2.00012228  2.99994449]
8 [ 1.00004964 -2.00003403  3.00000277]
x = [ 1.00004964 -2.00003403  3.00000277]
